In [1]:
from functools import cache
import heapq

In [2]:
data = open("input/22").read().splitlines()

In [3]:
depth = int(data[0].split(": ")[1])

In [4]:
x, y = data[1].split(": ")[1].split(",")
target = (int(x), int(y))

In [5]:
@cache
def get_type(pos):
    ero = get_erosion_level(pos)
    return ero % 3

In [6]:
@cache
def get_erosion_level(pos):
    geo = get_geologic_index(pos)
    return (geo + depth) % 20183

In [7]:
@cache
def get_geologic_index(pos):
    x, y = pos
    if x == 0 and y == 0:
        return 0

    if pos == target:
        return 0

    if y == 0:
        return x * 16807

    if x == 0:
        return y * 48271

    return get_erosion_level((x - 1, y)) * get_erosion_level((x, y - 1))

In [8]:
part1 = 0
for a in range(target[0] + 1):
    for b in range(target[1] + 1):
        part1 += get_type((a, b))  

In [9]:
print(f"Answer #1: {part1}")

Answer #1: 11843


# Part 2

Will try to use A*

In [10]:
def get_valid_tools(type_):
    if type_ == 0:
        return ["torch", "climbing_gear"]
    if type_ == 1:
        return ["climbing_gear", "None"]
    if type_ == 2:
        return ["torch", "None"]

In [11]:
def manhattan(x, y):
    return abs(x - target[0]) + abs(y - target[1])

In [12]:
q = []
start = (0, 0, "torch")
heapq.heappush(q, (0, 0, start))
visited = set()
while q:
    h_score, g_score, state = heapq.heappop(q)
    if state in visited:
        continue
    visited.add(state)
    
    x, y, equipment = state
    if (x, y) == target and equipment == "torch":
        break
    neighbours = [
        (x + 1, y),
        (x - 1, y),
        (x, y + 1),
        (x, y - 1),
    ]
    for nx, ny in neighbours:
        if nx < 0 or ny < 0:
            continue
        if (nx, ny, equipment) in visited:
            continue

        next_type = get_type((nx, ny))
        valid_tools = get_valid_tools(next_type)
        if equipment in valid_tools:
            new_g_score = g_score + 1
            f_score = new_g_score + manhattan(nx, ny)
            heapq.heappush(q, (f_score, new_g_score, (nx, ny, equipment)))

    # Switch tool
    for tool in get_valid_tools(get_type((x, y))):
        if tool == equipment:
            continue
        new_state = (x, y, tool)
        if new_state in visited:
            continue
        new_g_score = g_score + 7
        f_score = new_g_score + manhattan(x, y)
        heapq.heappush(q, (f_score, new_g_score, new_state))
    

In [13]:
print(f"Answer #2: {g_score}")

Answer #2: 1078
